In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import sys

# --- robust project root detection ---
project_root = Path.cwd()
while not (project_root / "src").exists() and project_root != project_root.parent:
    project_root = project_root.parent

src_path = project_root / "src"
fig_dir = project_root / "figures"
res_dir = project_root / "results"

fig_dir.mkdir(exist_ok=True)
res_dir.mkdir(exist_ok=True)

print("CWD:", Path.cwd())
print("project_root:", project_root)
print("src_path:", src_path, "exists:", src_path.exists())

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from model_proteostasis import (
    ModelParams,
    simulate_load,    
)


CWD: C:\Users\Regi\Documents\Replication-Asymmetry-Model\notebooks
project_root: C:\Users\Regi\Documents\Replication-Asymmetry-Model
src_path: C:\Users\Regi\Documents\Replication-Asymmetry-Model\src exists: True


In [3]:
def mean_misfold_per_step(asym: bool, p: ModelParams) -> float:
    if not asym:
        lam = p.G * p.MU_BASE
    else:
        lam = p.G * (
            p.f_E * (p.MU_LE_E + p.MU_LA_E)
            + p.f_L * (p.MU_LE_L + p.MU_LA_L)
        )
    return p.p_mis * p.Y * lam

def mu_star(asym: bool, d: float, p: ModelParams) -> float:
    mu_mis = mean_misfold_per_step(asym, p)
    return (1 - d) * (mu_mis + p.B) / d


In [7]:
p = ModelParams()

d_val = 0.8
gamma_val = 0.0
T = 500
reps = 1000

L0 = simulate_load(False, reps=reps, T=T, d=d_val, gamma=gamma_val, p=p, seed=111)
L1 = simulate_load(True,  reps=reps, T=T, d=d_val, gamma=gamma_val, p=p, seed=222)

mean0_sim = L0.mean()
mean1_sim = L1.mean()
delta_pct_sim = (mean1_sim - mean0_sim) / mean0_sim * 100.0

mu0_analytic = mu_star(False, d_val, p)
mu1_analytic = mu_star(True,  d_val, p)
delta_pct_analytic = (mu1_analytic - mu0_analytic) / mu0_analytic * 100.0

mean0_sim, mean1_sim, delta_pct_sim, mu0_analytic, mu1_analytic, delta_pct_analytic


(np.float64(505.19939945271517),
 np.float64(511.91458989837105),
 np.float64(1.3292158408997465),
 504.4999999999999,
 512.7199999999999,
 1.629335976214079)

In [8]:
rows = [{
    "d": d_val,
    "gamma": gamma_val,
    "reps": reps,
    "T": T,
    "mean_L0_sim": float(mean0_sim),
    "mean_L1_sim": float(mean1_sim),
    "delta_pct_sim": float(delta_pct_sim),
    "mu0_analytic": float(mu0_analytic),
    "mu1_analytic": float(mu1_analytic),
    "delta_pct_analytic": float(delta_pct_analytic),
}]

df_load = pd.DataFrame(rows)
out_path = res_dir / "load_dynamics_H0_H1.csv"
df_load.to_csv(out_path, index=False)
print("Saved:", out_path)


Saved: C:\Users\Regi\Documents\Replication-Asymmetry-Model\results\load_dynamics_H0_H1.csv


In [9]:
plt.figure()
bins = 50

plt.hist(L0, bins=bins, alpha=0.5, density=True, label="H0 (symmetric)")
plt.hist(L1, bins=bins, alpha=0.5, density=True, label="H1 (asymmetric)")

plt.axvline(L0.mean(), linestyle=":", label=f"H0 mean ≈ {L0.mean():.1f}")
plt.axvline(L1.mean(), linestyle="--", label=f"H1 mean ≈ {L1.mean():.1f}")

plt.xlabel("Final load L_T")
plt.ylabel("Density")
plt.title(f"Single-lineage load distribution (T={T}, d={d_val})")
plt.legend()

fig_path = fig_dir / "load_distribution_H0_vs_H1.png"
plt.savefig(fig_path, dpi=300, bbox_inches="tight")
plt.close()
print("Saved:", fig_path)


Saved: C:\Users\Regi\Documents\Replication-Asymmetry-Model\figures\load_distribution_H0_vs_H1.png
